In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("pyspark-testing").getOrCreate()

In [17]:
filepath = "./data/books3000.csv"
df = spark.read.options(
    header='True',
    inferSchema='True',
    delimiter=","
).csv(filepath)

In [14]:
type(df)

pyspark.sql.dataframe.DataFrame

In [18]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- small_image_url: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- books_count: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: double (nullable = true)
 |-- original_publication_year: integer (nullable = true)
 |-- original_title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- average_rating: double (nullable = true)



In [22]:
df.show(n=5, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------------------------------
 id                        | 51                                                                          
 author                    | Cassandra Clare                                                             
 title                     | City of Bones (The Mortal Instruments, #1)                                  
 image_url                 | https://images.gr-assets.com/books/1432730315m/256683.jpg                   
 small_image_url           | https://images.gr-assets.com/books/1432730315s/256683.jpg                   
 price                     | 1461                                                                        
 books_count               | 178                                                                         
 isbn                      | 1416914285                                                                  
 isbn13                    | 9.78142E12       

In [24]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show(truncate=False, vertical=True)

-RECORD 0------------------------
 id                        | 0   
 author                    | 0   
 title                     | 0   
 image_url                 | 0   
 small_image_url           | 0   
 price                     | 0   
 books_count               | 0   
 isbn                      | 0   
 isbn13                    | 0   
 original_publication_year | 0   
 original_title            | 0   
 language_code             | 0   
 average_rating            | 0   



In [26]:
df.select(count("id")).show()

+---------+
|count(id)|
+---------+
|     2999|
+---------+



In [32]:
df.select(when(isnan('price').alias("hello!"), True)).show()

+------------------------------------------------+
|CASE WHEN isnan(price) AS `hello!` THEN true END|
+------------------------------------------------+
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                                            null|
|                              

In [33]:
rating_metrics = {
    "high" = 5*0.75,
    "average" = 5*0.5,
    "low" = 5*0.25

}

SyntaxError: invalid syntax (<ipython-input-33-4594d9956205>, line 2)

In [ ]:
# average_rating
# 
# high rating